In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_columns', None)

## Load Excel to DataFrames

In [3]:
# Create a dictionary of DataFrames from each Excel file provided by ODA
files = {'licenses':'License','locations':'Location','violations':'Violation','inspections':'Contact','addresses':'Address','classifications':'LicenseClassification'}
# Change this based on where you are keeping latest version of data
path = 'BFS FOIA Request Excel Files/'

dfs = {}
for dfname in files:
    try:
        df = pd.ExcelFile(path + files[dfname] + '.xlsx')
    except:
        print "Failed to fetch Excel file."
    df = df.parse("Sheet1",na_values='')
    dfs.update({dfname:df})


## License processing

In [4]:
# Identify unique license type for each license from classifications table
license_types_from_classifications = dfs['classifications'].groupby(['_fkLicenseID','LicenseType']).ModifiedBy.count().reset_index()

# Add the license type to the licenses table
dfs['licenses'] = pd.merge(dfs['licenses'],license_types_from_classifications,left_on='_pkLicenseID',right_on='_fkLicenseID',how='left')

# Clean up column names to clarify the source of the license type info
dfs['licenses'] = dfs['licenses'].rename(columns={'LicenseType_x':'LicenseType_numeric','LicenseType_y':'LicenseType_from_classifications'})

# First try using the type listed from the Classification table
dfs['licenses']['LicenseTypeDescription'] = dfs['licenses']['LicenseType_from_classifications']

In [5]:
# Next try the alpha code from end of internal ODA license number

# First pull the last few characters of the license number, which in recent years ODA has used to indicate type
dfs['licenses']['Type_alphacode'] = dfs['licenses'].LicenseNumber.str.extract(r'[0-9]([A-Z]*)$')

# Then pull list of types
license_types_from_alpha = pd.read_csv('license-types-from-alpha.csv')
license_types_from_alpha = license_types_from_alpha.rename(columns={'Description':'LicenseType_from_alpha'})

# Eliminate superflous columns
license_types_from_alpha.drop([col for col in license_types_from_alpha.columns if 'Unnamed' in col],axis=1,inplace=True)

# Join them based on the license number code
dfs['licenses'] = pd.merge(dfs['licenses'],license_types_from_alpha,left_on='Type_alphacode',right_on='Alpha_code',how='left')

# If the classifications table didn't have the description we needed, used the one based on alpha code
criteria = dfs['licenses'].LicenseTypeDescription.isnull()==True
dfs['licenses'].loc[criteria,'LicenseTypeDescription'] = dfs['licenses'].LicenseType_from_alpha

In [6]:
# Now try joining based on the field LicenseType, which was used in early years but is no longer populated

# Pull list of types using numeric code
license_types_from_numeric = pd.read_csv('license-types-from-numeric.csv')
license_types_from_numeric = license_types_from_numeric.rename(columns={'Description':'LicenseType_from_numeric'})

# Eliminate superflous columns
license_types_from_numeric.drop([col for col in license_types_from_numeric.columns if 'Unnamed' in col],axis=1,inplace=True)

# Join the code description to the licenses table using the numeric code
dfs['licenses'] = dfs['licenses'].merge(license_types_from_numeric[pd.notnull(license_types_from_numeric.Numeric_code)],left_on='LicenseType_numeric',right_on='Numeric_code',how='left')

# If license type is still blank after other joins, use the version from numeric if available
dfs['licenses'].loc[dfs['licenses'].LicenseTypeDescription.isnull() == True,'LicenseTypeDescription'] = dfs['licenses'].LicenseType_from_numeric

# Fill nulls in License Types
dfs['licenses'].LicenseTypeDescription = dfs['licenses'].LicenseTypeDescription.fillna('Not listed')

# Created stripped-down license_ID dataframe with basic info
license_IDs = dfs['licenses'][['_pkLicenseID','_fkLocationID','LicenseTypeDescription','LicenseStatusSecondary']]

In [7]:
# Isolate a set of only licenses that are active

# Use the Secondary status, per Sarah's advice
active_licenses = license_IDs[(license_IDs.LicenseStatusSecondary == 'Active')]

# Create short list
active_license_IDs = active_licenses[['_pkLicenseID','LicenseTypeDescription','_fkLocationID']]

# Separate retail into new Dataframe
active_retail_licenses = active_licenses[active_licenses.LicenseTypeDescription=='Retail Food Establishment']
active_retail_license_IDs = active_retail_licenses[['_pkLicenseID','LicenseTypeDescription','_fkLocationID']]


## Address processing

In [8]:
# Isolate physical addresses from the addresses table
physical_addresses = dfs['addresses'][dfs['addresses'].AddressType.str.contains('hysical') == True]
physical_addresses['gz_ModTimeStamp'] = pd.to_datetime(physical_addresses['gz_ModTimeStamp'], errors='coerce')
physical_addresses = physical_addresses[physical_addresses['gz_ModTimeStamp'].isnull() == False]
# If there are multiple physical addresses for a location, use the one updated most recently
physical_addresses = physical_addresses.loc[physical_addresses.groupby('_fkLocationID').gz_ModTimeStamp.idxmax()]

/Users/ssuo1/OneDrive - Advance Local Media LLC/Code/food-safety/ENV/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## Location processing

In [10]:
# FIX ISSUE WITH CURLY APOSTROPHE IN STORE NAME
dfs['locations'].LocationName = dfs['locations'].LocationName.str.replace(u'\u2019',"'")


In [11]:
# Isolate retail locations with one active license
active_retail_location_IDs = active_retail_license_IDs.groupby('_fkLocationID').count().reset_index()
active_retail_location_IDs = active_retail_location_IDs.drop(columns=['LicenseTypeDescription','_pkLicenseID'])

In [12]:
# Eliminate locations that have specialized food processing along with a retail operation
# First grab all licenses associated with a retail location
active_all_licenses_retail_locations = pd.merge(active_license_IDs,active_retail_location_IDs,on='_fkLocationID',how='inner')
location_licenses = active_all_licenses_retail_locations[['_fkLocationID','LicenseTypeDescription']]
# Transpose license types into columns with 0-1 count for each column
location_licenses = pd.pivot_table(location_licenses,index='_fkLocationID',columns='LicenseTypeDescription',aggfunc=len).rename_axis(None,axis=1).reset_index()
location_licenses = location_licenses.fillna(0)
# Flag locations that don't have at least a bakery or meat sellers license
location_licenses['Bakery_or_meat'] = location_licenses['Bakery'] + location_licenses['Meat Sellers License']
location_licenses['Bk_Ms_flag'] = 'No'
location_licenses.loc[location_licenses['Bakery_or_meat'] > 0,'Bk_Ms_flag'] = 'Yes'
# Add up columns that are specialty food processing license types
#location_licenses['Processing_types']=location_licenses['Custom Meat Processor']+location_licenses['Custom Mobile Slaughter']+location_licenses['Custom Stationary Slaughter']+location_licenses['Dairy Products Plant']+location_licenses['Egg Handler']+location_licenses['Fluid Milk Producer']+location_licenses['Food Processing Establishment']+location_licenses['Milk Sampler-Grader']+location_licenses['Non-Processing Distributor Grade A']+location_licenses['Non-Slaughtering Processor']+location_licenses['Refrigerated Locker Plant']+location_licenses['Shellfish Distributor, Shipper, Wholesaler']+location_licenses['Shellfish Grower']+location_licenses['Shellfish Shucker/Packer']+location_licenses['Slaughterhouse']+location_licenses['Not listed']
location_licenses['Processing_types']=location_licenses['Custom Meat Processor']+location_licenses['Custom Mobile Slaughter']+location_licenses['Custom Stationary Slaughter']+location_licenses['Dairy Products Plant']+location_licenses['Egg Handler']+location_licenses['Fluid Milk Producer']+location_licenses['Food Processing Establishment']+location_licenses['Milk Sampler-Grader']+location_licenses['Non-Processing Distributor Grade A']+location_licenses['Refrigerated Locker Plant']+location_licenses['Shellfish Distributor, Shipper, Wholesaler']+location_licenses['Shellfish Grower']+location_licenses['Shellfish Shucker/Packer']+location_licenses['Slaughterhouse']+location_licenses['Not listed']
location_licenses['Processing_flag'] = 'No'
location_licenses.loc[location_licenses['Processing_types'] > 0,'Processing_flag'] = 'Yes'
location_licenses[['_fkLocationID','Processing_flag','Bk_Ms_flag']]
# Add flags to retail locations frame indicating limitations on licenses at location
active_retail_locations = pd.merge(dfs['locations'],location_licenses,left_on='_pkLocationID',right_on='_fkLocationID',how='inner')


In [13]:
# Flag locations that prepare raw food
raw = dfs['classifications'][(dfs['classifications'].LicenseType=='Retail Food Establishment')&
                       (dfs['classifications'].Classification1=='Packaged & Unpackaged Foods')&
                       (dfs['classifications'].Classification2=='Raw/Precooked')]
raw = raw.groupby(['_fkLicenseID']).ModifiedBy.count().reset_index()
raw = raw.drop(columns='ModifiedBy')
raw['Raw'] = 'Yes'

In [14]:
# Add the 'raw' flag to the active retail license IDs
active_retail_license_IDs_with_raw = pd.merge(active_retail_license_IDs,raw,left_on='_pkLicenseID',right_on='_fkLicenseID',how='left') ## If it's left join, then we end up with dupe
active_retail_license_IDs_with_raw = active_retail_license_IDs_with_raw.drop(columns=['_pkLicenseID','LicenseTypeDescription','_fkLicenseID'])
# Add this information to the retail locations list
active_retail_locations = pd.merge(active_retail_locations,active_retail_license_IDs_with_raw,left_on='_pkLocationID',right_on='_fkLocationID',how='left')
# Clean up
active_retail_locations = active_retail_locations.drop(columns='_fkLocationID_y')

In [15]:
# Select only raw food handling locations
active_retail_locations=active_retail_locations[(active_retail_locations['FoodService'].str.contains('Full')==True)|
                                                (active_retail_locations['Raw']=='Yes')]


In [16]:
# Eliminate places that also do food processing
active_retail_locations=active_retail_locations[active_retail_locations['Processing_flag']=='No']
# Strip it down to just an ID
active_retail_location_IDs = active_retail_locations[['_pkLocationID']]

In [17]:
# Add physical addresses to active retail locations
active_retail_locations = pd.merge(active_retail_locations,physical_addresses,left_on='_pkLocationID',right_on='_fkLocationID',how='left')
active_retail_locations = active_retail_locations.drop(columns=['_fkLocationID_x'])

In [18]:
# Extract from location name a chain name
chains = { "AFC Sushi":["Afc Sushi","AFC Sushi"],
          "Hissho Sushi":["Hissho"],
          "Snowfox":["Snowfox"],
          "Little Samurais":["Little Samurais"],
          "Kikka Sushi":["Kikka","KIKKA"],
          "FujiSan Sushi":["FujiSan","Fujisan"],
          "Stop and Go":["Stop and Go","Stop And Go","Stop & Go","Stop N Go","Stop n Go"],
          "US Market":["Us Market","US Market","U S Market"],
          "Walmart":["Wal-Mart","Wal Mart","Walmart"],
          "WinCo":["Winco","WinCo","Waremart"],
          "Arco ":["Arco ","ARCO ","AM/PM"],
          "TJ Maxx":["T J Maxx","TJ Maxx","Tj Maxx"],
          "TNT Market":["T N T Market","TNT Market","Tnt Market"],
          "Erickson's":["Erickson's","Ericksons"],
          "Lil' Pantry":["Lil Pantry","Lil' Pantry"],
          "Market of Choice":["Market Of Choice","Market of Choice"],
          "McKay's Market":["McKay's Market","Mckay's Market"],
          "7-Eleven":["7-Eleven","7 Eleven"],
          "Bi-Mart":["Bi Mart","Bi-Mart"],
          "Jackson's ":["Jacksons ","Jackson's "],
          "Buy 2":["Buy2","Buy 2"],
          "LTI/Milky Way":["LTI/Milky Way","LTI / Milky Way"],
          "Reser's":["Resers","Reser's"],
          "Franz Bakery":["Franz Bakery","Franz Family"],
          "Market of Choice":["Market of Choice","Market Of Choice"],
          "Marshall's":["Marshall's #","Marshalls #"],
          "Everyone's Market":["Everyone's Market","Everyones Market"],
          "A-1 Market":["A-1 Market","A1 Market"],
          "Sportsman's Warehouse":["Sportsman's Warehouse","Sportsmans Warehouse"],
          "Kmart":["K Mart","Kmart"],
          "Tillamook County Creamery Association":["Tillamook County Creamery","Tillamook Co Creamery"],
          "Hi School Pharmacy":["Hi School","Hi-School"],
          "Loves Travel Stop":["Loves Travel","Love's Travel"],
          "Swire Coca Cola":["Swire Coca Cola","Swire Coca-Cola"],
          "H Mart":["H Mart","H-Mart"],
          "Harry's Fresh Foods":["Harry's Fresh Foods","Harrys Fresh Foods"],
          "Zupan's":["Zupan's","Zupans"],
          "A-1 Market":["A-1 Market"],
          "Albertson's":["Albertson"],
          "Circle K":["Circle K"],
          "Costco":["Costco"],
          "Fred Meyer":["Fred Meyer"],
          "Little Samurai":["Little Samurai"],
          "New Seasons":["New Seasons"],
          "Ray's Food Place":["Ray's Food Place"],
          "Roth's":["Roth's"],
          "Safeway":["Safeway"],
          "Trader Joe's":["Trader Joe"],
          "Whole Foods":["Whole Foods"],
          "Papa Murphy's":["Papa Murphy"],
          "Plaid Pantry":["Plaid Pant"],
          "Chevron":["Chevron"],
          "Dollar Tree":["Dollar Tree","AM/PM"],
          "Schwan":["Schwan"],
          "Walgreens":["Walgreen"],
          "Rite Aid":["Rite Aid"],
          "Dollar General":["Dollar General"],
          "Grocery Outlet":["Grocery Outlet"],
          "Dari Mart":["Dari Mart"],
          "Davis Tran-Lite":["Davis Trans"],
          "Hood River Juice":["Hood River Juice"],
          "Ross Dress for Less":["Ross Dress"],
          "General Nutrition":["General Nutrition"],
          "Center Market":["Center Market"],
          "Figaro's":["Figaro"],
          "Frito Lay":["Frito Lay"],
          "Home Depot":["Home Depot"],
          "Target":["Target"],
          "Cash & Carry":["Cash & Carry"],
          "La Mota":["La Mota"],
          "New Seasons":["New Seasons"],
          "Mr. Nice Guy":["Nice Guy"],
          "Zwald Transport":["Zwald Transport"],
          "Big Lots":["Big Lots"],
          "Hickory Farms":["Hickory Farm"],
          "Office Depot":["Office Depot"],
          "Stop N Save":["Stop N Save"],
          "24 Hour Fitness":["24 Hour Fitness"],
          "Natural Grocers":["Natural Grocers"],
          "Space Age":["Space Age"],
          "Big 5 Sporting Goods":["Big 5"],
          "Great Harvest":["Great Harvest"],
          "Staples":["Staples"],
          "Bed Bath and Beyond":["Bed Bath"],
          "Bimbo Bakeries USA":["Bimbo"],
          "Made In Oregon":["Made In Oregon"],
          "Minute Market":["Minute Market"],
          "Roth's":["Roth's"],
          "Tuesday Morning":["Tuesday Morning"],
          "Columbia Distributing":["Columbia Distributing"],
          "JC Penney":["Jc Penney"],
          "Western Beverage":["Western Beverage"],
          "Astro":["Astro"],
          "Crown Market":["Crown Market"],
          "Franz Thrift Store":["Franz Thrift"],
          "Old Navy":["Old Navy"],
          "See's Candies":["See's"],
          "Main Street Market":["Main Street Market"],
          "Speedy Mart":["Speedy Mart"],
          "American Market":["American Market"],
          "Cost Plus":["Cost Plus"],
          "DS Services":["DS Services"],
          "Electric Lettuce":["Electric Lettuce"],
          "Henningsen Cold Storage":["Henningsen"],
          "Sherm's Thunderbird":["Sherm's"],
          "Vitamin Shoppe":["Vitamin Shoppe"],
          "Williams Bakery":["Williams Bakery"],
          "Americold":["Americold"],
          "Attis Trading":["Attis Trading"],
          "Edible Arrangements":["Edible Arrangements"],
          "Fast Mart":["Fast Mart"],
          "Oregon Bud Company":["Oregon Bud"],
          "Pacific Seafood":["Pacific Seafood"],
          "Paradies":["Paradies"],
          "Short Stop":["Short Stop"],
          "Sweet Relief":["Sweet Relief"],
          "Umpqua Dairy":["Umpqua Dair"],
          "Weight Watchers":["Weight Watchers"],
          "Cannabis Nation":["Cannabis Nation"],
          "Cash King":["Cash King"],
          "Chalice Farms":["Chalice Farms"],
          "Euphoria Chocolate":["Euphoria Chocolate"],
          "Farmer Brothers":["Farmer Brothers"],
          "Going Green":["Going Green"],
          "Grocery Depot":["Grocery Depot"],
          "Harry And David":["Harry And David"],
          "Kaya Shack":["Kaya Shack"],
          "Merritt #1":["Merritt #1"],
          "Nake Windery":["Nake Windery"],
          "OG Collective":["OG Collective"],
          "Quality Food Center":["Quality Food Center"],
          "Rocky Mountain Chocolate":["Rocky Mountain Chocolate"],
          "Rogue Creamery":["Rogue Creamery"],
          "Today's Herbal Choice":["Today's Herbal Choice"],
          "Toys R Us":["Toys R Us"],
          "Ama Mini Mart":["Ama Mini Mart"],
          "Bridgeport Distributing":["Bridgeport Distributing"],
          "CBD Hemp":["CBD Hemp"],
          "Columbia River Dairy":["Columbia River Dairy"],
          "Colvin Oil":["Colvin Oil"],
          "Complete Nutrition":["Complete Nutrition"],
          "Dayton Natural Meats":["Dayton Natural Meats"],
          "Desert Lake Technologies":["Desert Lake Technologies"],
          "Everyday Deals":["Everyday Deals"],
          "Green Knottz":["Green Knottz"],
          "Green Zebra":["Green Zebra"],
          "Hood River Distillers":["Hood River Distillers"],
          "Humm Kombucha":["Humm Kombucha"],
          "Jenny Craig":["Jenny Craig"],
          "Joann Fabric":["Joann Fabric"],
          "Klamath Algae Products":["Klamath Algae"],
          "Macy's":["Macy's"],
          "Market Place Fresh Foods":["Market Place Fresh Food"],
          "Meduri Farms":["Meduri Farm"],
          "Moo Lines Inc":["Moo Lines Inc"],
          "Moonstruck Chocolate":["Moonstruck Chocolate"],
          "Mountain Rose Herbs":["Mountain Rose Herbs"],
          "Norpac Foods":["Norpac Foods"],
          "Nothing Bundt Cake":["Nothing Bundt Cake"],
          "Oakshire Brewing":["Oakshire Brewing"],
          "OFD Foods":["OFD Foods"],
          "One Stop Mart":["One Stop Mart"],
          "Oregon Cherry Growers":["Oregon Cherry Growers"],
          "Point Blank Distributing":["Point Blank Distributing"],
          "Portland Farmers Market":["Portland Farmers Market"],
          "Quick Stop Market":["Quick Stop Market"],
          "Rogue Valley Cannabis":["Rogue Valley Cannabis"],
          "Savy's Sweets":["Savy's Sweets"],
          "Schwietert's Cones and Candy":["Schwietert"],
          "Smart Choice Market":["Smart Choice Market"],
          "Spring Valley Dairy":["Spring Valley Dairy"],
          "Stop N Shop":["Stop N Shop"],
          "Summit Foods":["Summit Foods"],
          "Sunrise Market":["Sunrise Market"],
          "Super Supplements":["Super Supplements"],
          "Sur La Table":["Sur La Table"],
          "Tea Chai Te":["Tea Chai Te"],
          "The Fit Foods":["The Fit Foods"],
          "Tobacco Town":["Tobacco Town"],
          "Tokyo Starfish":["Tokyo Starfish"],
          "Tri Valley Food Mart":["Tri Valley Food Mart"],
          "United Market":["United Market"],
          "Vitamin World":["Vitamin World"],
          "Western Oregon Dispensary":["Western Oregon Dispensary"],
          "Wheeler Dealer":["Wheeler Dealer"],
          "Willamette Beverage":["Willamette Beverage"],
          "Wymore Transfer Company":["Wymore Transfer Company"]}
import re
for chain,variants in chains.iteritems():
    for variant in variants:
        my_regex = r"(.*" + re.escape(variant) + r")"
        criteria = active_retail_locations.LocationName.str.match(my_regex)==True
        active_retail_locations.loc[criteria,'Chain'] = chain

# Different way of submitting literal regex to the search for chain names
specials = { "REI":["^Rei$","^Rei ","Recreational Equipment"],
            "76":["^76","76 "],
            "Ars Fresno":["^Ars "],
            "Home Goods":["^Home Goods "],
            "Neighborhood Market":["^Neighborhood Market"]
           }
for chain,variants in specials.iteritems():
    for variant in variants:
        my_regex = r"(" + variant + r")"
        criteria = active_retail_locations.LocationName.str.match(my_regex)==True
        active_retail_locations.loc[criteria,'Chain'] = chain

# Special way of submitting regex to exclude certain words
nots = {"Shell":["Shellfish"],"Nectar":["Pure Nectar","Nectar Creek"]}
for chain,variants in nots.iteritems():
    for variant in variants:
        my_regex = r"(" + re.escape(chain) + r")" + r"(?!" + variant + r")"
        criteria = active_retail_locations.LocationName.str.match(my_regex)==True
        active_retail_locations.loc[criteria,'Chain'] = chain


## Inspection processing

In [19]:
# Isolate only completed inspections with a date listed
dfs['inspections'].ContactDate = pd.to_datetime(dfs['inspections'].ContactDate, errors='coerce')
dfs['inspections'] = dfs['inspections'][dfs['inspections'].ContactDate.isnull() == False]
dfs['inspections'] = dfs['inspections'][dfs['inspections'].IsComplete == 1]

# Isolate only routine inspections
routine_inspections = dfs['inspections'][dfs['inspections'].ContactType == 'Routine']

# Isolate only the latest
latest_routine_inspections = routine_inspections.loc[routine_inspections.groupby('_fkLicenseID').ContactDate.idxmax()]
latest_routine_inspection_IDs = latest_routine_inspections[['_pkContactID','_fkLicenseID']]
latest_routine_inspection_IDs['Latest'] = 'Yes'

# Add flag to all inspection records to show whether it's the latest
dfs['inspections'] = pd.merge(dfs['inspections'],latest_routine_inspection_IDs,on=['_pkContactID','_fkLicenseID'],how='left')

/Users/ssuo1/OneDrive - Advance Local Media LLC/Code/food-safety/ENV/lib/python2.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [20]:
# Add information about the license type associated with the inspection
dfs['inspections'] = pd.merge(dfs['inspections'],license_IDs,left_on='_fkLicenseID',right_on='_pkLicenseID',how='left')

In [21]:
# If no license type is available, then borrow from "contact type" to populate license type. 
dfs['inspections'].loc[dfs['inspections'].LicenseTypeDescription.isnull()==True,'LicenseTypeDescription'] = dfs['inspections'].ContactType
dfs['inspections'].loc[dfs['inspections'].LicenseTypeDescription=='Not listed','LicenseTypeDescription'] = dfs['inspections'].ContactType

In [22]:
# Isolate only inspections for active licenses
active_all_licenses_retail_locations = pd.merge(active_license_IDs,active_retail_location_IDs,left_on='_fkLocationID',right_on='_pkLocationID',how='inner')
active_all_licenses_retail_locations = active_all_licenses_retail_locations[['_pkLicenseID']]
active_retail_inspections = pd.merge(active_all_licenses_retail_locations,dfs['inspections'],left_on='_pkLicenseID',right_on='_fkLicenseID',how='inner')
active_retail_inspection_IDs = active_retail_inspections[['_pkContactID','_fkLocationID_x','_fkLicenseID',"LicenseTypeDescription"]]
active_retail_inspection_IDs = active_retail_inspection_IDs.rename(columns={'_fkLocationID_x':'_fkLocationID'})

active_latest_retail_inspections = active_retail_inspections[active_retail_inspections.Latest == 'Yes']
active_latest_retail_inspections = active_latest_retail_inspections[active_latest_retail_inspections.LicenseStatusSecondary == 'Active']
active_latest_retail_inspection_IDs = active_latest_retail_inspections[['_pkContactID','_fkLocationID_x','_pkLicenseID_x',"LicenseTypeDescription"]]
active_latest_retail_inspection_IDs = active_latest_retail_inspection_IDs.rename(columns={'_fkLocationID_x':'_fkLocationID','_pkLicenseID_x':'_pkLicenseID'})

# Per Sarah's advice, dropping prepackaged meat sellers because their violations are noted in RFE inspection
active_latest_retail_inspection_IDs = active_latest_retail_inspection_IDs[(active_latest_retail_inspection_IDs.LicenseTypeDescription!='Pre-packaged only- Meat sellers')&(active_latest_retail_inspection_IDs.LicenseTypeDescription!='Prepackaged Meat Sellers')]
# Same with full inspections table
active_retail_inspections = active_retail_inspections[(active_retail_inspections.LicenseTypeDescription!='Pre-packaged only- Meat sellers')&(active_retail_inspections.LicenseTypeDescription!='Prepackaged Meat Sellers')]

# Keep only the inspection ID
active_latest_retail_inspection_IDs = active_latest_retail_inspections[['_pkContactID']]


## Compute data on violations

In [23]:
# Make the points field be a number
dfs['violations'].Points = pd.to_numeric(dfs['violations'].Points, errors='coerce')
# Add flag for high risk violations: 5-point violations with a Risk Factor
criteria = (dfs['violations'].Points == 5) & (dfs['violations'].RiskFactor.isnull() == False) & (dfs['violations'].RiskFactor.str.match(r'^\\n$')==False)
dfs['violations'].loc[criteria,'HighRisk'] = 'Yes'
# Clean the Repeat field, converting '0 1' to 1 | checking with Sarah on this
criteria = dfs['violations'].Repeat == '0 1'
dfs['violations'].loc[criteria,'Repeat'] = '1'
# Convert Repeat nulls to 0
criteria = dfs['violations'].Repeat.isnull()==True
dfs['violations'].loc[criteria,'Repeat'] = 0
# Make Repeat field a number
dfs['violations'].Repeat = pd.to_numeric(dfs['violations'].Repeat, errors='coerce')


In [24]:
###############################################################
# Fix the point values on retail locations that are active
###############################################################

# First, isolate violations found at active retail locations
license_IDs_for_active_retail_locations = pd.merge(license_IDs,active_retail_location_IDs,left_on='_fkLocationID',right_on='_pkLocationID',how='right')
license_IDs_for_active_retail_locations = license_IDs_for_active_retail_locations[license_IDs_for_active_retail_locations.LicenseStatusSecondary=='Active']
license_IDs_for_active_retail_locations = license_IDs_for_active_retail_locations[['_pkLicenseID','_fkLocationID']]
active_retail_violations = pd.merge(dfs['violations'],license_IDs_for_active_retail_locations,left_on='_fkLicenseID',right_on='_pkLicenseID',how='right')
# Now store the original value in a different column name
active_retail_violations = active_retail_violations.rename(columns={'Points':'Original_point_values'})
active_retail_violations['Points'] = ''
active_retail_violations.Points = pd.to_numeric(active_retail_violations.Points, errors='coerce')
# Finally, update the point values based on Criticality Code, which is a consistent identifier when applied to retail 
criteria = active_retail_violations.CriticalityCode == 'C'
active_retail_violations.loc[criteria,'Points'] = 0
criteria = active_retail_violations.CriticalityCode == 'Pf'
active_retail_violations.loc[criteria,'Points'] = 3
criteria = active_retail_violations.CriticalityCode == 'P'
active_retail_violations.loc[criteria,'Points'] = 5


In [25]:
# To allow for inspections that found 0 violations, join violations with all inspections
#active_retail_violations = pd.merge(active_retail_inspections,active_retail_violations,left_on='_pkContactID',right_on='_fkContactID',how='left')
active_retail_violations = pd.merge(active_retail_inspection_IDs,active_retail_violations,left_on='_pkContactID',right_on='_fkContactID',how='left')

# Convert null points to 0 points
active_retail_violations.loc[active_retail_violations['_fkViolationID'].isnull()==True,'Points'] = 0

In [26]:
active_retail_violations.drop([col for col in active_retail_violations.columns if '_y' in col],axis=1,inplace=True)


In [27]:
active_retail_violations = active_retail_violations.rename(columns= {'_fkLocationID_x':'_fkLocationID','_fkLicenseID_x':'_fkLicenseID'})

In [28]:
# Isolate violations found in LATEST inspections on active license
active_latest_retail_violations = pd.merge(active_latest_retail_inspection_IDs,active_retail_violations,left_on='_pkContactID',right_on='_fkContactID',how='inner')

# Isolate high-risk violations found in latest inspections on active license
active_latest_risky_retail_violations = active_latest_retail_violations[active_latest_retail_violations['HighRisk']=='Yes']

# Clean up column names
active_latest_retail_violations.drop([col for col in active_latest_retail_violations.columns if '_y' in col],axis=1,inplace=True)
active_latest_retail_violations = active_latest_retail_violations.drop(columns=['_pkContactID_x'])
active_latest_risky_retail_violations.drop([col for col in active_latest_risky_retail_violations.columns if '_y' in col],axis=1,inplace=True)
active_latest_risky_retail_violations = active_latest_risky_retail_violations.drop(columns=['_pkContactID_x'])


In [29]:
# Tabulate total points deducted in latest inspection(s) at each location
active_latest_violation_total_points = active_latest_retail_violations.groupby('_fkLocationID').Points.sum().reset_index()
active_latest_violation_total_points = active_latest_violation_total_points.rename(columns={'Points':'Latest_deductions'})

# Tabulate average points deducted from license
active_latest_violation_total_by_license = active_latest_retail_violations.groupby(['_fkLocationID','_fkLicenseID','LicenseTypeDescription']).Points.sum().reset_index()
active_latest_violation_average_points = active_latest_violation_total_by_license.groupby(['_fkLocationID']).Points.mean().reset_index()
active_latest_violation_average_points = active_latest_violation_average_points.rename(columns={'Points':'Latest_deductions_average'})

# Tabulate total points deducted from RETAIL license in latest inspection(s) at each location
active_latest_retail_violation_total_points = active_latest_violation_total_by_license[active_latest_violation_total_by_license['LicenseTypeDescription']=='Retail Food Establishment']
active_latest_retail_violation_total_points = active_latest_retail_violation_total_points.rename(columns={'Points':'Latest_retail_deductions'})
active_latest_retail_violation_total_points = active_latest_retail_violation_total_points[['_fkLocationID','Latest_retail_deductions']]

# Tabulate total number of repeat violations in latest inspection(s) at each location
active_latest_repeat_violation_counts = active_latest_retail_violations.groupby(['_fkLocationID']).Repeat.sum().reset_index()
active_latest_repeat_violation_counts = active_latest_repeat_violation_counts.rename(columns={'Repeat':'Repeat_violations'})

# Tabulate total number of high risk violations in latest inspection at each location
active_latest_risky_violation_counts = active_latest_risky_retail_violations.groupby(['_fkLocationID'])._pkViolationID.count().reset_index()
active_latest_risky_violation_counts = active_latest_risky_violation_counts.rename(columns={'_pkViolationID':'High_risk_violations'})

In [30]:
# Join these data points onto location info for retail, active only
facilities = pd.merge(active_retail_locations,active_latest_violation_total_points,left_on='_pkLocationID',right_on='_fkLocationID',how='left')
facilities = pd.merge(facilities,active_latest_violation_average_points,left_on='_pkLocationID',right_on='_fkLocationID',how='left')
facilities = pd.merge(facilities,active_latest_retail_violation_total_points,left_on='_pkLocationID',right_on='_fkLocationID',how='left')
facilities = pd.merge(facilities,active_latest_repeat_violation_counts,left_on='_pkLocationID',right_on='_fkLocationID',how='left')
facilities = pd.merge(facilities,active_latest_risky_violation_counts,left_on='_pkLocationID',right_on='_fkLocationID',how='left')



In [31]:
# Final cleanup of facilities table
facilities = facilities.drop(columns=['_fkLocationID_x','_fkLocationID_y','_fkL2KPersonID'])
facilities.Latest_deductions = facilities.Latest_deductions.fillna(0)
facilities.Repeat_violations = facilities.Repeat_violations.fillna(0)
facilities.High_risk_violations = facilities.High_risk_violations.fillna(0)

# Drop unneeded columns with licensing 0-1 flags
facilities = facilities.drop(facilities.columns[44:65],axis=1)
facilities = facilities.drop(columns=['Processing_types'])

# Output facilities file for Dave
facilities.to_csv('facilities.csv',header=True, index=False, encoding='utf-8')

In [32]:
# Output facilities file for Dave
facilities.to_csv('facilities.csv',header=True, index=False, encoding='utf-8')

In [33]:
# Make remaining base files for Dave

# Clean up inspections
active_retail_inspections = active_retail_inspections.rename(columns={'_fkLocationID_x':'_fkLocationID','LicenseTypeDescription_x':'LicenseTypeDescription'})
# Export
active_retail_inspections.to_csv('inspections.csv',header=True,index=False,encoding='utf-8')

# Clean up violations
active_retail_violations = active_retail_violations[active_retail_violations._pkViolationID.isnull()==False]
active_retail_violations = active_retail_violations.drop(columns=['_pkLicenseID','_fkLocationID'])
# Export
active_retail_violations.to_csv('violations.csv', encoding="utf-8", index=False)